# Day 4: Passport Processing

https://adventofcode.com/2020/day/4

## Part 1

In [44]:
from pathlib import Path

INPUTS = Path('input.txt').resolve().read_text().strip()
# Because the inputs are separated by a blank line, we can reliably
# split the passport chunks by the double newline '\n\n'.
INPUTS = INPUTS.split('\n\n')
# The results will all contain newline characters in the middle of their contents,
# so we just replace those with spaces to clean them up.
INPUTS = [x.replace('\n', ' ') for x in INPUTS]

# Lay out the required fields here
REQUIRED_FIELDS = ('byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid')

num_valid = 0
for passport in INPUTS:
    # Begin by splitting the content, which by default is on spaces.
    content = passport.split()

    # As each of the keys are exactly 3 characters long, and every
    # entry takes the same format, we can just chop off the data
    # with a simple slice. Additionally, we don't really need the 'cid'
    # key, as that's optional; so we'll conditionally exclude it.
    content = [x[:3] for x in content if x[:3] != 'cid']

    # Now that we have a list of the keys excluding the optional 'cid',
    # turns out we don't really need to check that they match.
    # Also, we can guess they're unique for each entry.

    # So, all that's really needed is to check that we have as many keys
    # as our list of required keys:
    if len(content) == len(REQUIRED_FIELDS):
        num_valid += 1

# And that dumb little check produces our correct output
print(f"{num_valid} valid passports")


256 valid passports


## Part 2

In [49]:
# Now things get more complicated. Time for real data validation.
# Let's set up a validator to help us out:

import re

hcl_pattern = re.compile(r"^#[0-9a-f]{6}$")
pid_pattern = re.compile(r"^\d{9}$")
hgt_pattern = re.compile(r"^(?P<measure>\d{2,3})(?P<unit>in|cm)$")

def is_valid(entry: dict) -> bool:
    
    def has_required_fields() -> bool:
        nonlocal entry
        fields = [k for k in entry.keys() if k != 'cid']
        return len(fields) == len(REQUIRED_FIELDS)
    
    if not has_required_fields():
        return False
    
    # That being covered, create validators for each of the parts of the passport:

    def is_valid_byr(entry) -> bool:
        return 1920 <= int(entry['byr']) <= 2002

    def is_valid_iyr(entry) -> bool:
        return 2010 <= int(entry['iyr']) <= 2020

    def is_valid_eyr(entry) -> bool:
        return 2020 <= int(entry['eyr']) <= 2030

    def is_valid_hgt(entry) -> bool:
        match = hgt_pattern.match(entry['hgt'])
        if not match:
            return False
        match_dict = match.groupdict()
        measure = int(match_dict['measure'])
        if match_dict['unit'] == 'cm':
            return 150 <= measure <= 193
        return 59 <= measure <= 76

    def is_valid_hcl(entry) -> bool:
        return bool(hcl_pattern.match(entry['hcl']))

    def is_valid_ecl(entry) -> bool:
        accepted = ('amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth')
        return entry['ecl'] in accepted

    def is_valid_pid(entry) -> bool:
        return bool(pid_pattern.match(entry['pid']))
    
    # Now run then and return all() for the checks
    return all([
        is_valid_byr(entry),
        is_valid_iyr(entry),
        is_valid_eyr(entry),
        is_valid_hgt(entry),
        is_valid_hcl(entry),
        is_valid_ecl(entry),
        is_valid_pid(entry),
    ])

# Now a converter to turn our original passports into dicts we can actually use
# (doing the work we skipped last time)
def passport_to_dict(passport: str) -> dict:
    output = {}
    for part in passport.split():
        key, val = part.split(':')
        output[key] = val
    return output


# Now we can do the work:
valid_count = 0
for passport in INPUTS:
    passport_dict = passport_to_dict(passport)
    if is_valid(passport_dict):
        valid_count += 1

print(f"{valid_count} valid passports.")

198 valid passports.
